# 0. Extra Config as my pc has multiple JVMs 

In [ ]:
!/usr/libexec/java_home -V

Matching Java Virtual Machines (8):
    23 (arm64) "Eclipse Adoptium" - "OpenJDK 23" /Library/Java/JavaVirtualMachines/temurin-23.jdk/Contents/Home
    19.0.2 (arm64) "Amazon.com Inc." - "Amazon Corretto 19" /Users/him/Library/Java/JavaVirtualMachines/corretto-19.0.2/Contents/Home
    17.0.8 (arm64) "Oracle Corporation" - "Java SE 17.0.8" /Library/Java/JavaVirtualMachines/jdk-17.jdk/Contents/Home
    11.0.23 (arm64) "Amazon.com Inc." - "Amazon Corretto 11" /Users/him/Library/Java/JavaVirtualMachines/corretto-11.0.23/Contents/Home
    11.0.22 (arm64) "Eclipse Adoptium" - "OpenJDK 11.0.22" /Library/Java/JavaVirtualMachines/temurin-11.jdk/Contents/Home
    1.8.431.10 (x86_64) "Oracle Corporation" - "Java" /Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home
    1.8.0_412 (arm64) "Amazon" - "Amazon Corretto 8" /Users/him/Library/Java/JavaVirtualMachines/corretto-1.8.0_412/Contents/Home
    1.8.0_412 (x86_64) "Amazon" - "Amazon Corretto 8" /Users/him/Library/Java/JavaVirtualMach

In [ ]:
import os

In [ ]:
java_path = "/Library/Java/JavaVirtualMachines/jdk-17.jdk/Contents/Home"
java_home = os.environ.get('JAVA_HOME', None)

print("Existing Java Home: ",java_home)

if (not java_home) or (java_path not in java_home):
    os.environ['JAVA_HOME'] = java_path

print("Updated Java Home: ",os.environ.get('JAVA_HOME', None))

Existing Java Home:  None
Updated Java Home:  /Library/Java/JavaVirtualMachines/jdk-17.jdk/Contents/Home


In [ ]:
# print java version

os.system('java -version')

java version "17.0.8" 2023-07-18 LTS
Java(TM) SE Runtime Environment (build 17.0.8+9-LTS-211)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.8+9-LTS-211, mixed mode, sharing)


0

# 1. Load data

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('../data/train.csv')

# 2. Prepocessing

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

In [ ]:
train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train_df['Sex'] = train_df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
train_df['Embarked'] = train_df['Embarked'].apply(lambda x: 0 if x == 'S' else 1)

In [ ]:
spark = SparkSession\
        .builder\
        .appName("ensemble")\
        .getOrCreate()


24/11/28 18:09:24 WARN Utils: Your hostname, Yuen-Man-Hims-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.89.166.58 instead (on interface en0)
24/11/28 18:09:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 18:09:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
data = spark.createDataFrame(train_df)
train, valid = data.randomSplit([0.5, 0.5], seed=12345)

assembler = VectorAssembler(
    inputCols = train_df.columns[1:].to_list(),
    outputCol = "features")
train = assembler.transform(train)
valid = assembler.transform(valid)
y_true = valid.select('Survived').toPandas()

# 3. Model

In [ ]:
from sklearn.metrics import f1_score

## 3.1 Naive-bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
# create the trainer and set its parameters
nb = NaiveBayes(
    smoothing=1.0, 
    modelType="complement", 
    labelCol="Survived"
    )

# train the model
nbModel = nb.fit(train)

# select example rows to display.
nbResult = nbModel.transform(valid)

24/11/28 18:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/11/28 18:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [ ]:
nbResult.show()

+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex|              Age|SibSp|Parch|   Fare|Embarked|            features|       rawPrediction|         probability|prediction|
+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|             19.0|    3|    2|  263.0|       0|[1.0,1.0,19.0,3.0...|[-102.90479354434...|[2.03711713726180...|       1.0|
|       0|     1|  1|29.69911764705882|    0|    0|27.7208|       1|[1.0,1.0,29.69911...|[-0.1374212834842...|[0.87160295684474...|       0.0|
|       0|     1|  1|             40.0|    0|    0|27.7208|       1|[1.0,1.0,40.0,0.0...|[-0.0021174196878...|[0.99788482046380...|       0.0|
|       0|     1|  1|             42.0|    1|    0|   52.0|       0|[1.0,1.0,42.0,1.0...|[-2.7403122235727...|[0.06455018965571...|       1.0|

In [ ]:
nb_y_pred = nbResult.select('prediction').toPandas().values
nb_f1 = f1_score(y_true, nb_y_pred)

## 3.2 Linear Support Vector Machine (LinearSVM)

In [ ]:
from pyspark.ml.classification import LinearSVC

In [ ]:
lsvc = LinearSVC(maxIter=10, regParam=0.1, labelCol="Survived", featuresCol="features")

# Fit the model
lsvcModel = lsvc.fit(train)

lsvcResult = lsvcModel.transform(valid)

In [ ]:
lsvcResult.show()

+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+----------+
|Survived|Pclass|Sex|              Age|SibSp|Parch|   Fare|Embarked|            features|       rawPrediction|prediction|
+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+----------+
|       0|     1|  1|             19.0|    3|    2|  263.0|       0|[1.0,1.0,19.0,3.0...|[1.11859142297071...|       0.0|
|       0|     1|  1|29.69911764705882|    0|    0|27.7208|       1|[1.0,1.0,29.69911...|[0.88578780970810...|       0.0|
|       0|     1|  1|             40.0|    0|    0|27.7208|       1|[1.0,1.0,40.0,0.0...|[0.89721772738040...|       0.0|
|       0|     1|  1|             42.0|    1|    0|   52.0|       0|[1.0,1.0,42.0,1.0...|[1.06649657993054...|       0.0|
|       0|     1|  1|             47.0|    0|    0|   52.0|       0|[1.0,1.0,47.0,0.0...|[0.94127411707960...|       0.0|
|       0|     1|  1|   

In [ ]:
lsvc_y_pred = lsvcResult.select('prediction').toPandas().values
lsvc_f1 = f1_score(y_true, lsvc_y_pred)

## 3.3 Multilayer Perceptron (MLP)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
trainer = MultilayerPerceptronClassifier(
    labelCol="Survived",
    featuresCol="features",
    maxIter=1500, 
    layers=[7, 32, 64, 16, 2], 
    blockSize=64, 
    seed=1234
    )

# train the model
mlpModel = trainer.fit(train)

# compute accuracy on the test set
mlpResult = mlpModel.transform(valid)

In [ ]:
mlpResult.show()

+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex|              Age|SibSp|Parch|   Fare|Embarked|            features|       rawPrediction|         probability|prediction|
+--------+------+---+-----------------+-----+-----+-------+--------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|             19.0|    3|    2|  263.0|       0|[1.0,1.0,19.0,3.0...|[0.33219551564158...|[0.72767999677926...|       0.0|
|       0|     1|  1|29.69911764705882|    0|    0|27.7208|       1|[1.0,1.0,29.69911...|[0.33219802082379...|[0.72768097704230...|       0.0|
|       0|     1|  1|             40.0|    0|    0|27.7208|       1|[1.0,1.0,40.0,0.0...|[0.33219601623760...|[0.72768020779275...|       0.0|
|       0|     1|  1|             42.0|    1|    0|   52.0|       0|[1.0,1.0,42.0,1.0...|[0.33222122790630...|[0.72768988259470...|       0.0|

In [ ]:
mlp_y_pred = mlpResult.select('prediction').toPandas().values
mlp_f1 = f1_score(y_true, mlp_y_pred)

# 4. Ensemble of 3 models (Naive-bayes, LinearSVM, MLP)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score

In [ ]:
f1_total = nb_f1 + lsvc_f1 + mlp_f1
y_pred = np.round((nb_y_pred * nb_f1 + lsvc_y_pred * lsvc_f1 + mlp_y_pred * mlp_f1)/f1_total)

# Create accuracy score
print("accuracy: ", accuracy_score(y_true, y_pred))

# Create classification report
print(classification_report(y_true, y_pred))

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 5. Produce the submission file

In [ ]:
test_df = pd.read_csv('../data/test.csv')

test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_df['Sex'] = train_df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
test_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Embarked'] = train_df['Embarked'].apply(lambda x: 0 if x == 'S' else 1)

In [ ]:
test = assembler.transform(spark.createDataFrame(test_df))

In [ ]:
nb_pred = nbModel.transfor(test).select('prediction').toPandas().values
lsvc_pred = lsvcModel.transfor(test).select('prediction').toPandas().values
mlp_pred = mlpModel.transfor(test).select('prediction').toPandas().values

y_pred = np.round((nb_pred * nb_f1 + lsvc_pred * lsvc_f1 + mlp_pred * mlp_f1)/f1_total)

In [ ]:
np.savetxt('../result/submission.csv', y_pred, delimiter=',', fmt='%d')